 ## <b>Connecting vxi11 compatible devices via Ethernet </b>

In [10]:
!pip3 install python-vxi11

In [1]:
import vxi11
import time

In [2]:
keysight_ip = '192.168.0.8'

In [3]:
Keysight = vxi11.Instrument(keysight_ip)
print(Keysight.ask("*IDN?"))

Agilent Technologies,33512B,MY57401183,4.00-1.19-2.00-58-00


In [4]:
rigol_ip = '192.168.0.14'

In [5]:
Rigol = vxi11.Instrument(rigol_ip)
print(Rigol.ask("*IDN?"))

RIGOL TECHNOLOGIES,DS1074Z,DS1ZA193816619,00.04.04.SP3


## <b> Interacting with a Brüel & Kjær Type 3050-B-6 using Python </b>

In [102]:
cd ../PyBnK

C:\Users\ricky\Documents\VM3000-Microphones\PyBnK


In [7]:
from bnk.bnk import WavHeader, OpenWav, Instrument

In [9]:
bnk_ip = '192.168.0.9'
ADAC = Instrument(bnk_ip)
print(ADAC) # Show some info about the BnK device

ConnectionError: HTTPConnectionPool(host='192.168.0.9', port=80): Max retries exceeded with url: /rest/rec/onchange?last=0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000019DD47E9688>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

## <b> Python Wrapper for Keysight 33500B - Modified from Carlin's code</b>

In [6]:
class keysight():
    
    def __init__(self,instrument_ID=keysight_ip):
        self.Keysight =  vxi11.Instrument(instrument_ID)
        print(self.Keysight.ask("*IDN?"))
        
    def enable(self):
        self.Keysight.write("OUTPut ON")
        
    def disable(self):
        self.Keysight.write("OUTPut OFF") 
        
    def enable_high_impedance(self):
        self.Keysight.write("OUTPut:LOAD INF")  
        
    def sine(self,frequency=500,amplitude=0.01,offset=0):
        self.Keysight.write(f"VOLTage:OFFSet {offset}")
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write(f"FREQuency {frequency}")
        
    def whitenoise(self, bandwidth=1500, amplitude=0.01):
        self.Keysight.write("VOLTage:UNIT VRMS")
        self.Keysight.write(f"VOLTage {amplitude}")
        self.Keysight.write("FUNCtion NOISe")
        self.Keysight.write(f"FUNCtion:NOISe:BANDwidth {bandwidth}")
        
    def enable_burst(self,frequency=2000,ncycles=50,int_period=5):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write("BURSt:STATe ON")
        self.Keysight.write(f"FREQuency {frequency}")
        self.Keysight.write(f"BURSt:NCYCles {ncycles}")
        self.Keysight.write(f"BURSt:INTernal:PERiod {int_period}")          
        self.Keysight.write("OUTPut ON")
        
    def disable_burst(self):
        self.Keysight.write("BURSt:STATe OFF")
        self.Keysight.write("OUTPut OFF")
        
    def enable_sweep(self, mode="logarithmic", runTime=10):
        self.Keysight.write("FUNCtion SINusoid")
        self.Keysight.write("SWEep:STATe ON")
        self.Keysight.write(f"SWEep:SPAcing {mode}")
        self.Keysight.write(f"SWEep:TIME {runTime}")
        self.Keysight.write("OUTPut ON")
        
    def disable_sweep(self):
        self.Keysight.write("SWEep:STATe OFF")
        self.Keysight.write("OUTPut OFF")

## <b> Connecting to RPI Zero W </b>

In [7]:
#Connecting to the RPi over Wi-Fi
import datetime
from paramiko import SSHClient
from scp import SCPClient
RPi = '192.168.0.6' # This is the IP address when RPi is connected to SHL network, note that this changes depending on different networks unless static IP is set
ssh = SSHClient()
ssh.load_system_host_keys()
ssh.connect(RPi,username='pi')

In [8]:
scp = SCPClient(ssh.get_transport())

In [9]:
cd ../Audio Files

C:\Users\ricky\Documents\VM3000-Microphones\Audio Files


In [10]:
# Checking Current File directory

import os
print(os.getcwd())
#scp.get('test_file.txt')

C:\Users\ricky\Documents\VM3000-Microphones\Audio Files


In [11]:
# Checking RPI Zero W files using ls

stdin, stdout, stderr = ssh.exec_command('ls -l')
for x in stdout:
    print(x)

total 488744

-rw-r--r--  1 pi   pi     2704044 Sep  7 09:28 48000

-rw-r--r--  1 pi   pi         370 Sep 22 10:40 blinkatest.py

-rw-r--r--  1 pi   pi      960044 Mar  3 13:09 burstTest_20210312_094957.wav

-rw-r--r--  1 pi   pi      960044 Mar  3 13:09 burstTest_20210312_095048.wav

-rwxr-xr-x  1 pi   pi        2677 Aug 26  2020 i2smic.sh

lrwxrwxrwx  1 pi   pi          55 Sep 23 02:54 linux -> /home/pi/linux-988cc7beacc150756c3fbe40646afcf8438b741b

drwxr-xr-x 27 pi   pi        4096 Sep 22 16:18 linux-43857965e5f526b9df807e543102a11fac1c0bcc

-rw-r--r--  1 pi   pi   163445874 Sep 22 16:13 linux-43857965e5f526b9df807e543102a11fac1c0bcc.tar.gz

drwxr-xr-x 27 pi   pi        4096 Sep 23 02:58 linux-988cc7beacc150756c3fbe40646afcf8438b741b

-rw-r--r--  1 pi   pi   163419588 Sep 23 02:52 linux-988cc7beacc150756c3fbe40646afcf8438b741b.tar.gz

drwxr-xr-x 27 pi   pi        4096 Aug 26  2020 linux-fe2c7bf4cad4641dfb6f12712755515ab15815ca

-rw-r--r--  1 pi   pi   163526392 Aug 26  2020 linux-f

In [65]:
client = SSHClient()
client.load_system_host_keys()
client.connect('192.168.0.6',username='pi')
stdin, stdout, stderr = client.exec_command('touch test_file.txt')
print(stdin)

<paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0xd4c91908 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>


## <b> Class for Test Functions </b>

In [28]:
class signalTests():    
    def sweepTest(self, mode="logarithmic", runTime=10, sleepTime=5):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime
        import time
        
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 48000

        cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} sweepTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)
        
        testName = "Sweep Test"
        
        sg.enable_sweep(mode, runTime)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)
        sg.disable_sweep()

        print(f"Finished {testName} Output")
        
        time.sleep(15)
        
        scp.get(f'sweepTest_{now}.wav')
        
    def sineTest(self, frequency=2000, amplitude=0.01, offset=0, runTime=10, sleepTime=5):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 48000

        cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} sineTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)
        
        testName = "Sine Wave Test"
        
        sg.sine(frequency,amplitude,offset)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)

        print(f"Finished {testName} Output")
        
        scp.get(f'sineTest_{now}.wav')
        
    def whiteNoiseTest(self, bandwidth=500, amplitude=0.01, runTime=10, sleepTime=2):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 48000

        cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} whiteNoiseTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)      
        
        testName = "White Noise Test"
        
        sg.whitenoise(bandwidth,amplitude)
        sg.enable()
        time.sleep(runTime)
        sg.disable()
        time.sleep(sleepTime)

        print(f"Finished {testName} Output")
        
        scp.get(f'whiteNoiseTest_{now}.wav')
        
    def burstTest(self, frequency=2000, ncycles=10, interval=1):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 48000

        cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} burstTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)     
        
        testName = "Burst Test"

        sg.enable_burst(frequency,ncycles,interval)

        for i in range(1,ncycles,1):
            print(f'Pulse number {i}')
            Keysight.write(f"FREQuency {frequency}")
            sg.enable()
            time.sleep(interval)
            sg.disable()
            time.sleep(interval)

        print(f"Finished {testName} Output")  
        
        scp.get(f'burstTest_{now}.wav')
        
    def ambientTest(self):
        from paramiko import SSHClient
        from scp import SCPClient
        import datetime

        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        sample_rate = 48000

        cmd = f'arecord -D plughw:1 -c2 -r {sample_rate} -f S16_LE -t wav -V stereo -v -d {runTime} ambientTest_{now}.wav'

        print(cmd)
        ssh.exec_command(cmd)   
        
        testName = "Ambient Test"
        
        print(f"Finished {testName} Output")
        
        scp.get(f'ambientTest_{now}.wav')

## <b> Test Cases </b>

In [13]:
sg=keysight()
sg.enable_high_impedance()

Agilent Technologies,33512B,MY57401183,4.00-1.19-2.00-58-00


In [ ]:
import paramiko
import time
import datetime

In [78]:
# Sweep Test

test1_name = "Sweep Test"
mode = "logarithmic"
sweep_period = 5

sg.enable_sweep(mode, sweep_period)
sg.enable()
time.sleep(sweep_period)
sg.disable()
time.sleep(sweep_period)
sg.disable_sweep()

print(f"Finished {test1_name} Output")

Finished Sweep Test Output


In [13]:
# Continuous Sine Wave Test

test2_name = "Sine Wave Test"
frequency = 2000
amplitude = 0.01 # Vrms
offset = 0
sleep_time = 2

sg.sine(frequency,amplitude,offset)
sg.enable()
time.sleep(sleep_time)
sg.disable()
time.sleep(sleep_time)

print(f"Finished {test2_name} Output")

Finished Sine Wave Test Output


In [376]:
# White Noise Test

test3_name = "White Noise Test"
bandwidth = 500
amplitude = 0.01 # Vrms
sleep_time = 2

sg.whitenoise(bandwidth,amplitude)
sg.enable()
time.sleep(sleep_time)
sg.disable()s
time.sleep(sleep_time)

print(f"Finished {test3_name} Output")

Finished


In [16]:
# Burst Sweep Test

test4_name = "Burst Sweep Test"
frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)
  
for f in range(2000,5000,1000):
    print(f'Pulsing at {f} Hz')
    Keysight.write(f"FREQuency {f}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)

sg.disable_burst()
    
print(f"Finished {test4_name} Output")

Pulsing at 2000 Hz
Pulsing at 3000 Hz
Pulsing at 4000 Hz
Finished Burst Sweep Test Output


In [17]:
# Burst Test Constant Frequency

test5_name = "Constant Frequency Burst Test"
frequency = 2000
ncycles = 10
int_period = 1
sleep_time = 2

sg.enable_burst(frequency,ncycles,int_period)

for i in range(1,11,1):
    print(f'Pulse number {i}')
    Keysight.write(f"FREQuency {frequency}")
    sg.enable()
    time.sleep(sleep_time)
    sg.disable()
    time.sleep(sleep_time)
          
print(f"Finished {test5_name} Output")

Pulse number 1
Pulse number 2
Pulse number 3
Pulse number 4
Pulse number 5
Pulse number 6
Pulse number 7
Pulse number 8
Pulse number 9
Pulse number 10
Finished Constant Frequency Burst Test Output


## <b> Frequency Response Characteristics Testing Script </b>

In [29]:
st = signalTests()

In [ ]:
# Running Frequency Sweep Test & recording audio from RPI Zero W 

mode="logarithmic"
runTime=5
sleepTime=2

st.sweepTest(mode,runTime,sleepTime)

arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 5 sweepTest_20210312_105329.wav


In [16]:
# Running Sine Wave Test & recording audio from RPI Zero W 

frequency=2000
amplitude=0.01 # Vrms
offset=0
runTime=5
sleepTime=2

st.sineTest(frequency,amplitude,offset,runTime,sleepTime)

arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 5 sineTest_20210312_103918.wav
Finished Sine Wave Test Output


In [17]:
# Running White Noise Test & recording audio from RPI Zero W

bandwidth=500
amplitude=0.01 # Vrms
runTime=5
sleepTime=2

st.whiteNoiseTest(bandwidth,amplitude,runTime,sleepTime)

arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 5 whiteNoiseTest_20210312_103932.wav
Finished White Noise Test Output


In [21]:
# Running Burst Test & recording audio from RPI Zero W

frequency=2000
ncycles=5
interval=1

st.burstTest(frequency,ncycles,interval)

arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 5 burstTest_20210312_104154.wav
Pulse number 1
Pulse number 2
Pulse number 3
Pulse number 4
Finished Burst Test Output


In [22]:
# Running Ambient Noise Test & recording audio from RPI Zero W

st.ambientTest()

arecord -D plughw:1 -c2 -r 48000 -f S16_LE -t wav -V stereo -v -d 5 ambientTest_20210312_104319.wav
Finished Ambient Test Output
